In [2]:
import torch
import torch.nn as nn
import torch.optim as optim



In [3]:
class MLP(nn.Module):
    def __init__(self, hidden=4):
        super().__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)  # logits (no sigmoid!)

def train_gate(name, x, y, epochs=2000, lr=0.1, hidden=4, seed=0, device=None, verbose=False):
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    torch.manual_seed(seed)
    x, y = x.to(device), y.to(device)

    model = MLP(hidden=hidden).to(device)
    loss_fn = nn.BCEWithLogitsLoss()
    opt = optim.Adam(model.parameters(), lr=lr)

    for ep in range(epochs):
        opt.zero_grad()
        logits = model(x)
        loss = loss_fn(logits, y)
        loss.backward()
        opt.step()
        if verbose and (ep + 1) % max(1, epochs // 10) == 0:
            print(f"[{name}] epoch {ep+1}/{epochs} loss={loss.item():.4f}")

    with torch.no_grad():
        logits = model(x)
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()
        acc = (preds == y).float().mean().item()
        print(f"\n{name} GATE (acc={acc*100:.1f}%)")
        print("Input\tProb\tPred\tActual")
        for i in range(len(x)):
            print(f"{x[i].tolist()}\t{probs[i].item():.3f}\t{int(preds[i].item())}\t{int(y[i].item())}")
    return model

# Input patterns (truth-table inputs)
x_data = torch.tensor([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=torch.float32)

# Truth tables
gates = {
    "AND": torch.tensor([[0], [0], [0], [1]], dtype=torch.float32),
    "OR":  torch.tensor([[0], [1], [1], [1]], dtype=torch.float32),
    "NOR": torch.tensor([[1], [0], [0], [0]], dtype=torch.float32)
}

# Train & evaluate each
models = {}
for name, y in gates.items():
    models[name] = train_gate(name, x_data, y, epochs=2000, lr=0.1)


AND GATE (acc=100.0%)
Input	Prob	Pred	Actual
[0.0, 0.0]	0.000	0	0
[0.0, 1.0]	0.000	0	0
[1.0, 0.0]	0.000	0	0
[1.0, 1.0]	1.000	1	1

OR GATE (acc=100.0%)
Input	Prob	Pred	Actual
[0.0, 0.0]	0.000	0	0
[0.0, 1.0]	1.000	1	1
[1.0, 0.0]	1.000	1	1
[1.0, 1.0]	1.000	1	1

NOR GATE (acc=100.0%)
Input	Prob	Pred	Actual
[0.0, 0.0]	1.000	1	1
[0.0, 1.0]	0.000	0	0
[1.0, 0.0]	0.000	0	0
[1.0, 1.0]	0.000	0	0
